In [1]:
import geopandas as gpd
import pandas as pd
import geopandas
import shapely.geometry as geom
from polygon_geohasher.polygon_geohasher import polygon_to_geohashes, geohashes_to_polygon
import pygeohash as gh
%matplotlib inline
import geohash
import geopandas as gp
import pandas as pd
import math
import geojson
from geojson import MultiLineString
from shapely.geometry import MultiLineString
from shapely.geometry import MultiPoint
import numpy as np
from shapely import geometry
from shapely.geometry import Point, Polygon, box,LineString
from geopandas import datasets, GeoDataFrame, read_file
from geopandas.tools import overlay
from matplotlib import pyplot
import matplotlib.pyplot as plt
from shapely.geometry.polygon import LinearRing, Polygon
from numpy import cos,sin,arccos
import numpy as np
from pylab import *
from shapely.ops import cascaded_union
from shapely.geometry import Point, LineString, mapping
import scipy.interpolate
import geopy
import geopy.distance
import time
import networkx as nx
from sklearn.model_selection import train_test_split
import datetime as dt

In [2]:
processed_geohash = pd.read_csv('processed_csv/processed_geohash.csv')

In [3]:
processed_geohash.head()

,trj_id,geohash_source,geohash_dest,timestamp_source,timestamp_dest,time_taken,hour,day
0,100,w21zthv,w21ztht,2019-04-16 02:44:15,2019-04-16 02:44:22,7.0,2,1
1,100,w21ztht,w21zthm,2019-04-16 02:44:22,2019-04-16 02:44:32,10.0,2,1
2,100,w21zthm,w21zthk,2019-04-16 02:44:32,2019-04-16 02:44:37,5.0,2,1
3,100,w21zthk,w21zthh,2019-04-16 02:44:37,2019-04-16 02:44:41,4.0,2,1
4,100,w21zthh,w21zt5u,2019-04-16 02:44:41,2019-04-16 02:44:50,9.0,2,1


In [4]:
processed_geohash['node_trip'] = processed_geohash['geohash_source'] + '-' + processed_geohash['geohash_dest']

In [5]:
len(unique(processed_geohash['node_trip']))

40759

In [6]:
processed_geohash.sort_values(by=['node_trip'])

,trj_id,geohash_source,geohash_dest,timestamp_source,timestamp_dest,time_taken,hour,day,node_trip
1019082,42292,w21xjy3,w21xjy1,2019-04-15 07:07:42,2019-04-15 07:07:49,7.0,7,0,w21xjy3-w21xjy1
14899,10452,w21xjy9,w21xjy3,2019-04-21 10:40:40,2019-04-21 10:40:45,5.0,10,6,w21xjy9-w21xjy3
1019081,42292,w21xjy9,w21xjy3,2019-04-15 07:07:37,2019-04-15 07:07:42,5.0,7,0,w21xjy9-w21xjy3
14898,10452,w21xjyd,w21xjy9,2019-04-21 10:40:38,2019-04-21 10:40:40,2.0,10,6,w21xjyd-w21xjy9
1019080,42292,w21xjyd,w21xjy9,2019-04-15 07:07:35,2019-04-15 07:07:37,2.0,7,0,w21xjyd-w21xjy9
...,...,...,...,...,...,...,...,...,...
2016680,65026,w23bhd3,w23bhd2,2019-04-19 03:31:00,2019-04-19 03:31:08,8.0,3,4,w23bhd3-w23bhd2
2016676,65026,w23bhd3,w23bhd9,2019-04-19 03:27:18,2019-04-19 03:27:27,9.0,3,4,w23bhd3-w23bhd9
2016679,65026,w23bhd9,w23bhd3,2019-04-19 03:30:39,2019-04-19 03:31:00,21.0,3,4,w23bhd9-w23bhd3
2016677,65026,w23bhd9,w23bhdd,2019-04-19 03:27:27,2019-04-19 03:27:57,30.0,3,4,w23bhd9-w23bhdd


In [7]:
busy_hours = [7, 8, 9, 17, 18, 19]

In [8]:
weekend_num = [5,6]
weekend_geohash = processed_geohash[processed_geohash['day'].isin(weekend_num)]
#weekend_visuals = pd.DataFrame(weekend_visuals.groupby('hour').mean().reset_index())
#weekend_visuals.head(24)

In [9]:
weekday_num = [0,1,2,3,4]
weekday_geohash = processed_geohash[processed_geohash['day'].isin(weekday_num)]


In [14]:
weekend_geohash_busy = weekend_geohash[weekend_geohash['hour'].isin(busy_hours)]
weekend_geohash_normal = weekend_geohash[~weekend_geohash['hour'].isin(busy_hours)]
weekday_geohash_busy = weekday_geohash[weekday_geohash['hour'].isin(busy_hours)]
weekday_geohash_normal = weekday_geohash[~weekday_geohash['hour'].isin(busy_hours)]

In [15]:
weekend_geohash_busy.head()

,trj_id,geohash_source,geohash_dest,timestamp_source,timestamp_dest,time_taken,hour,day,node_trip
581,10011,w21xx8v,w21xx8y,2019-04-20 08:06:26,2019-04-20 08:06:27,1.0,8,5,w21xx8v-w21xx8y
582,10011,w21xx8y,w21xx8z,2019-04-20 08:06:27,2019-04-20 08:06:37,10.0,8,5,w21xx8y-w21xx8z
583,10011,w21xx8z,w21xx8x,2019-04-20 08:06:37,2019-04-20 08:06:45,8.0,8,5,w21xx8z-w21xx8x
584,10011,w21xx8x,w21xxbb,2019-04-20 08:06:45,2019-04-20 08:06:46,1.0,8,5,w21xx8x-w21xxbb
585,10011,w21xxbb,w21xxb8,2019-04-20 08:06:46,2019-04-20 08:06:47,1.0,8,5,w21xxbb-w21xxb8


In [17]:
weekend_geohash_busy.to_csv('weekend_geohash_busy.csv', index=False)
weekend_geohash_normal.to_csv('weekend_geohash_normal.csv', index=False)
weekday_geohash_busy.to_csv('weekday_geohash_busy.csv', index=False)
weekday_geohash_normal.to_csv('weekday_geohash_normal.csv', index=False)

In [11]:
sg_geofabrik = pd.read_csv('sg_geofabrik.csv')
sg_geofabrik.head()

,geohash,no_poi,maxspeed
0,w21xjfc,1.0,70.0
1,w21xjzu,1.0,NaN
2,w21xnyn,3.0,NaN
3,w21xpnd,1.0,NaN
4,w21xpye,1.0,NaN


In [12]:
apr_weather = pd.read_csv('APR_SG_hourly_weather.csv')
apr_weather['datetime'] = pd.to_datetime(apr_weather['date'], format='%Y%m%d')
apr_weather['day'] = apr_weather['datetime'].dt.dayofweek
apr_weather['hour'] = apr_weather['time'].str.split(":").str[0]
apr_weather['hour'] = pd.to_numeric(apr_weather['hour'])
apr_weather.head()

,date,time,weather_string,rain,datetime,day,hour
0,20190401,1:00,passing clouds.,False,2019-04-01,0,1
1,20190401,2:00,thundershowers. partly cloudy.,True,2019-04-01,0,2
2,20190401,3:00,thunderstorms. passing clouds.,True,2019-04-01,0,3
3,20190401,4:00,thunderstorms. passing clouds.,True,2019-04-01,0,4
4,20190401,5:00,thunderstorms. passing clouds.,True,2019-04-01,0,5


In [15]:
print(len(weekend_geohash))
print(len(sg_geofabrik))

1108355
74946
